%%latex
\begin{verbtaim}
from rest_framework import serializers
class MovieSerializers(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    name = serializers.CharField(max_length=50)
    description = serializers.CharField(max_length=200)
    active = serializers.BooleanField(default=True)
\end{verbtaim}

from .models import Movie
from .serializers import MovieSerializers
from rest_framework.response import Response


from rest_framework import status
from rest_framework.views import APIView
# Create your views here.
class movie_list(APIView):
    serializer_class = MovieSerializers
    def get(self, request, format=None):
        print('================')
        print("movie_list")
        print('method:', request.method)
        print('================')
        movies = Movie.objects.all()
        serializer = self.serializer_class(movies, many=True)
        return Response(serializer.data)
        # without using serializer
        # data = {
        #     'movies': list(movies.values())
        #     }
        # return JsonResponse(data)
    def post(self, request, format=None):
        serializer = self.serializer_class(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        else:
            return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)       


class movie_details(APIView):
    serializer_class = MovieSerializers
    def get(self, request, pk):
        try:
            movie = Movie.objects.get(pk=pk)
        except Movie.DoesNotExist:
            return Response({'Error': 'Does Not Exist'}, status=status.HTTP_404_NOT_FOUND)
        serializer = self.serializer_class(movie)
        return Response(serializer.data)
        # data = {
        #     'name': movie.name,
        #     'description': movie.description,
        #     'active': movie.active
        #     }
        # return JsonResponse(data)
    def put(self, request, pk):
        movie = Movie.objects.get(pk=pk)
        serializer = self.serializer_class(movie, data=request.data, partial=True)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        else:
            return Response(serializer.errors)
    def delete(self, request, pk):
        movie = Movie.objects.get(pk=pk)
        movie.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)